In [1]:
%%time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.append("../")
from helpers.iaml01cw2_helpers import load_FashionMNIST

Xtrn, Ytrn, Xtst, Ytst = load_FashionMNIST("C:\\Users\\User\\Documents\\3. Third Year\\IAML\\IAML-CW2\\data")

Xtrn_orig = Xtrn.copy()
Xtst_orig = Xtst.copy()

Xtrn = Xtrn/255.0
Xtst = Xtst/255.0

Xmean = Xtrn.mean(0)

Xtrn_nm = Xtrn - Xmean
Xtst_nm = Xtst - Xmean

Wall time: 1.74 s


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC

C:\Users\User\anaconda3\envs\py3iaml\lib\site-packages\sklearn\utils\__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Sequence


In [ ]:
%%time

lgr_model = LogisticRegression()
lgr_model.fit(Xtrn_nm, Ytrn)

In [ ]:
%%time

lgr_pred = lgr_model.predict(Xtst_nm)
lgr_cm = confusion_matrix(Ytst, lgr_pred)

lgr_acc = np.trace(lgr_cm) / Ytst.shape[0]

In [ ]:
%%time

svc_model = SVC()
svc_model.fit(Xtrn_nm, Ytrn)

In [ ]:
%%time

svc_pred = svc_model.predict(Xtst_nm)
svc_cm = confusion_matrix(Ytst, svc_pred)

svc_acc = np.trace(svc_cm) / Ytst.shape[0]

In [ ]:
lgr_cm, lgr_acc, svc_cm, svc_acc

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(2)
pca.fit(Xtrn_nm)

In [ ]:
sigma_1, sigma_2 = pca.explained_variance_[:2]
xcoords = np.linspace(-5*sigma_1, 5* sigma_1, 100)
ycoords = np.linspace(-5*sigma_2, 5* sigma_2, 100)

xx, yy = np.meshgrid(xcoords, ycoords)

pred_data = pca.inverse_transform(np.c_[xx.ravel(), yy.ravel()])

In [ ]:
%%time

lgr_plot_pred = lgr_model.predict(pred_data).reshape(xx.shape)

plt.contourf(xx, yy, lgr_plot_pred, cmap=plt.cm.get_cmap('coolwarm', 10))
plt.colorbar()
plt.xlabel("1st Principal Component")
plt.ylabel("2nd Principal Component")

In [ ]:
%%time

svc_plot_pred = svc_model.predict(pred_data).reshape(xx.shape)

plt.contourf(xx, yy, svc_plot_pred, cmap=plt.cm.get_cmap('coolwarm', 10))
plt.colorbar()
plt.xlabel("1st Principal Component")
plt.ylabel("2nd Principal Component")

In [ ]:
small_indices = np.array([])
for c in range(10):
    class_indices = np.where(Ytrn == c)[0][:1000]
    small_indices = np.append(small_indices, class_indices).astype("int32")

In [ ]:
Xsmall = Xtrn_nm[small_indices]
Ysmall = Ytrn[small_indices]

In [ ]:
Xsmall.shape, Ysmall.shape

In [ ]:
from sklearn.model_selection import KFold

cvals = np.logspace(-2, 3, 10)
cvals

In [ ]:
%%time

kfold = KFold(3, True, 1)
cmeans = []


for c in cvals:
    acc_sum = 0
    for train, test in kfold.split(Xsmall, Ysmall):
        model = SVC(C = c, kernel = "rbf", gamma = "auto")
        model.fit(Xsmall[train], Ysmall[train])
        acc_sum += model.score(Xsmall[test], Ysmall[test])
    cmeans.append(acc_sum/3)

In [ ]:
plt.plot(cvals, cmeans)
plt.xscale("log")
plt.xlabel("C")
plt.ylabel("Mean Classification Accuracy")

In [ ]:
maxC_index = np.argmax(cmeans)
maxC_value = cvals[maxC_index]
cmeans[maxC_index], maxC_value

In [ ]:
optimal_model = SVC(C = maxC_value)
model.fit(Xtrn_nm, Ytrn)
print(model.score(Xtrn_nm, Ytrn))
print(model.score(Xtst_nm, Ytst))